In [ ]:
!wget https://www.lamsade.dauphine.fr/~cazenave/project2022.zip
!unzip project2022.zip

In [ ]:
import numpy as np
import keras
from keras import layers, regularizers
import gc
import golois

In [ ]:
planes = 31
moves = 361
N = 10000


input_data = np.random.randint(2, size=(N, 19, 19, planes))
input_data = input_data.astype ('float32')

policy = np.random.randint(moves, size=(N,))
policy = keras.utils.to_categorical (policy)

value = np.random.randint(2, size=(N,))
value = value.astype ('float32')

end = np.random.randint(2, size=(N, 19, 19, 2))
end = end.astype ('float32')

groups = np.zeros((N, 19, 19, 1))
groups = groups.astype ('float32')

print ("getValidation", flush = True)
golois.getValidation (input_data, policy, value, end)

In [3]:
def model_archi(filters=32, planes=31):

  input = keras.Input(shape=(19, 19, planes), name='board')
  x = layers.Conv2D(filters, 1, activation='relu', padding='same')(input)

  for i in range (9):

    x1 = layers.DepthwiseConv2D((7,7), padding='same', use_bias=False)(x)
    x1 = layers.LayerNormalization()(x1)
    x1 = layers.Conv2D(4*filters, 1, padding='same', activation='gelu')(x1)
    x1 = layers.Conv2D(filters, 1, padding='same')(x1)
    x = layers.add([x1, x])

  x = layers.BatchNormalization()(x)

  y = layers.Conv2D(1, 1, activation='relu', padding='same', use_bias = False, kernel_regularizer=regularizers.l2(0.0001))(x)
  y = layers.Flatten()(y)
  policy_head = layers.Activation('softmax', name='policy')(y)

  GlobalAveragePooling2D = layers.GlobalAveragePooling2D()(x)
  value_head = layers.Dense(50, activation='relu', kernel_regularizer=regularizers.l2(0.0001))(GlobalAveragePooling2D)
  value_head = layers.Dense(24, activation='relu', kernel_regularizer=regularizers.l2(0.0001))(value_head)
  value_head = layers.Dense(1, activation='sigmoid', name='value', kernel_regularizer=regularizers.l2(0.0001))(value_head)

  model = keras.Model(inputs=input, outputs=[policy_head, value_head])
  return model

In [4]:
from tensorflow.keras.utils import plot_model

# Show the model architecture
def plot_model_archi(model=model_archi()):
  return plot_model(model, show_shapes=True, show_layer_names=True)

In [ ]:
# model training

epochs = 100
batch = 32



def model_train_eval(model=model_archi(), epochs=100,batch =32,lr =0.0005):
  model.compile(optimizer=keras.optimizers.AdamW(learning_rate=lr),
                loss={'policy': 'categorical_crossentropy', 'value': 'binary_crossentropy'},
                loss_weights={'policy' : 1.0, 'value' : 1.0},
                metrics={'policy': 'categorical_accuracy', 'value': 'mse'})

  for i in range (1, epochs + 1):
      print ('epoch ' + str (i))
      golois.getBatch (input_data, policy, value, end, groups, i * N)
      history = model.fit(input_data,
                          {'policy': policy, 'value': value},
                          epochs=1, batch_size=batch)


      if (i % 5 == 0):
          gc.collect ()
      if (i % 20 == 0):
          golois.getValidation (input_data, policy, value, end)
          val = model.evaluate (input_data,
                                [policy, value], verbose = 0, batch_size=batch)
          print ("val =", val)
          model.save ('model_val_%s.h5'%str((val[-2])))


In [9]:
# model evaluation
model_name = "FADIL_BOUZKKA_4_4.h5"
model = keras.models.load_model(model_name)
score = model.evaluate(input_data,
                              {'policy': policy, 'value': value})
print("Model : ",model_name)
for i in range(len(model.metrics_names)):
  print("Metric ",model.metrics_names[i],":",str((score[i])))


313/313 [==============================] - 7s 18ms/step - loss: 2.4033 - policy_loss: 1.8591 - value_loss: 0.5377 - policy_categorical_accuracy: 0.4759 - value_mse: 0.0569
Model :  FADIL_BOUZKKA_4_4.h5
Metric  loss : 2.4032535552978516
Metric  policy_loss : 1.8591134548187256
Metric  value_loss : 0.5376906394958496
Metric  policy_categorical_accuracy : 0.47589999437332153
Metric  value_mse : 0.05694003403186798


In [10]:
#Testing all models (printing the accuracy):
from pathlib import Path

def test_models():

  p = Path("")
  Files = [str(f) for f in p.glob("*.h5")]

  for i in Files:
      name = i
      aa =keras.models.load_model(name)
      score = aa.evaluate(input_data,
                                        {'policy': policy, 'value': value})
      print("Metric for :%s"%str(Path(i).name),aa.metrics_names[-2],":",str((score[-2])))

test_models()

313/313 [==============================] - 7s 19ms/step - loss: 2.4033 - policy_loss: 1.8591 - value_loss: 0.5377 - policy_categorical_accuracy: 0.4759 - value_mse: 0.0569
Metric for :FADIL_BOUZKKA_4_4.h5 policy_categorical_accuracy : 0.47589999437332153
313/313 [==============================] - 5s 15ms/step - loss: 2.4439 - policy_loss: 1.8782 - value_loss: 0.5596 - policy_categorical_accuracy: 0.4714 - value_mse: 0.0653
Metric for :FADIL_BOUZKKA_4.h5 policy_categorical_accuracy : 0.4713999927043915
